In [1]:
import numpy as np

from syft.lib.adp.scalar import PhiScalar
from syft.lib.adp.entity import Entity
from syft.lib.adp.tensor import Tensor
from syft.lib.adp.adversarial_accountant import AdversarialAccountant
from syft.lib.adp.publish import publish

# Data Owner Loads in Private Data

In [47]:
x = PhiScalar(0,0.01,1, entity=Entity(unique_name="Bob Jones"))#|.name('Bob Age').send(domain)
y = PhiScalar(0,0.02,1).name('John Age').send(domain)
z = PhiScalar(0,0.02,1).name('Sue Age').send(domain)

In [ ]:
entities = [Entity(unique_name="Tudor"), Entity(unique_name="Madhava"), Entity(unique_name="Kritika"), Entity(unique_name="George")]

a = Tensor(np.array([[1,1],[1,0],[0,1],[0,0]])).private(min_val=0, max_val=1, entities=entities, is_discrete=True)
a = a.var('a').send(duet)

b = Tensor(np.array([[1],[1],[0],[0]])).private(min_val=0, max_val=1, entities=entities, is_discrete=False)
b = b.var('b').send(duet)

# Data Scientist Connects with Domain (with new key)

Behind the scenes, when a data scientist queries a private tensor, if that verification key doesn't yet have an accountant - we need to initailize one for that person.

In [ ]:
# duet = sy.duet()
x = duet.store['Bob Age']
y = duet.store['John Age']
z = duet.store['Sue Age']

In [ ]:
o = x*x + y*y + z
z = o * o * o

In [ ]:
z.get() # triggers an error 

In [ ]:
z_public = z.publish(sigma=0.1)
# BEHIND THE SCENES... it should try to find domain.accountants[verify_key] to 
# pass into the actual private method

In [ ]:
z_public.get() # will download automatically (no matter what)

# Local Demo Code

In [41]:
import numpy as np

from syft.lib.adp.scalar import PhiScalar
from syft.lib.adp.entity import Entity
from syft.lib.adp.tensor import Tensor
from syft.lib.adp.adversarial_accountant import AdversarialAccountant
from syft.lib.adp.publish import publish

In [42]:
# str(self.ssid) + "_" + str(self.entity.id).split(" ")[1][:-1]

In [43]:
x = PhiScalar(0,0.01,1) #.tag('x').send(duet)
y = PhiScalar(0,0.02,1) #.tag('y').send(duet)
z = PhiScalar(0,0.02,1) #.tag('z').send(duet)

In [44]:


p = x * x

o = p + y*y + z
z = o * o * o

In [45]:
acc = AdversarialAccountant(max_budget=10)
x_pub = z.publish(acc=acc, sigma=0.2)

/Users/andrewliamtrask/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/optimize.py:2371: RuntimeWarning: invalid value encountered in double_scalars
  w = xb - ((xb - xc) * tmp2 - (xb - xa) * tmp1) / denom


[-0.0373986135394829]

In [6]:
# the data scientist should never see this
acc = AdversarialAccountant(max_budget=10)

publish([z,z], acc=acc, sigma=0.2)

[-0.289880508368386, -0.319446255506735]

In [1]:
import numpy as np

from syft.lib.adp.scalar import PhiScalar
from syft.lib.adp.entity import Entity
from syft.lib.adp.tensor import Tensor
from syft.lib.adp.adversarial_accountant import AdversarialAccountant
from syft.lib.adp.publish import publish

acc = AdversarialAccountant(max_budget=3000000)


entities = [Entity(unique_name="Tudor"), Entity(unique_name="Madhava"), Entity(unique_name="Kritika"), Entity(unique_name="George")]

x = Tensor(np.array([[1,1],[1,0],[0,1],[0,0]])).private(min_val=0, max_val=1, entities=entities, is_discrete=True)
y = Tensor(np.array([[1],[1],[0],[0]])).private(min_val=0, max_val=1, entities=entities, is_discrete=False)

_weights = Tensor(np.random.uniform(size=(2,1)))

In [7]:
weights = _weights + 0
# acc = AdversarialAccountant(max_budget=7)  # causes error at end of budget
acc = AdversarialAccountant(max_budget=3000000)  

for i in range(2):
    batch_loss = 0

    pred = x.dot(weights)
    pre_loss = np.square(y-pred)
    loss = np.mean(pre_loss, axis=1)
    ledger = loss.backward(accumulate_grads=True)
    weight_grad = (weights.grad)
    weight_grad = weight_grad.publish(acc=acc, sigma=0.5)

    weights = weights - weight_grad
#     ledger.zero_grads()
    batch_loss += loss.value
    
#     print(np.sum(loss*loss).value)
    
acc.print_ledger()


<IntermediatePhiScalar: (-1.3481190154796612 < 0 < 2.198304850715509)>
found one!
<IntermediatePhiScalar: (-0.6466532377441928 < 0 < 2.198304850715509)>
found one!
<IntermediatePhiScalar: (-0.7396815582666453 < 0 < 3.28177459115145)>
found one!
<IntermediatePhiScalar: (-0.518177278879448 < 0 < 3.28177459115145)>
found one!
<Entity:George>	0.0


In [12]:
x = PhiScalar(0,0.01,1)

In [13]:
out = x * x

In [14]:
out.publish(acc=acc)

<IntermediatePhiScalar: (0.0 < 0.000100000000000000 < 1.0)>
found one!


[0.834445492996146]

In [10]:
acc.print_ledger()

<Entity:George>	0.0
<Entity:Richard_Alvarez>	0.0002737137892581691


In [1]:
x.entity

NameError: name 'x' is not defined

In [7]:
import sympy as sym

In [9]:
a,b = sym.symbols('a,b')

In [10]:
y = a + b

In [38]:
from sympy.core.numbers import Number
class Float2(Number):
    
    def __init__(self, val):
        self.val = val
        
    def __add__(self, other):
        print("adding")
        return Float(self.val - other.val)

In [39]:
out = y.subs({a:Float2(3)})
out

b + 3

In [40]:
out.subs({b:Float2(2)})

5

In [9]:
# loss[0].value.poly

IndexError: too many indices for array

24ffce1fe8ab44a583abecf0b2e1ca69_c06e7e291fdb4cb6b5ec0b9abbabf1bf + 3*3175e5ebc4944304bb4a1a010fa5003d_162afbde9a87417a9a49c14568ea1c11 + 4*5ec1e3788c314340a6ea45342375281e_162afbde9a87417a9a49c14568ea1c11 + 6*68ed3172c1f6417faa4f5ca24611e12d_d59083b02ef74bdf8122e0c51454aa71 + 6*98d310c205b743be9af907ff4646238b_b63effdec6ba47bf93a126d0b5cc7086 + d082ac7660e84d0c806997c2dbe4f600_b63effdec6ba47bf93a126d0b5cc7086 + 5*eeffa282abe44326ae5fc1b8a6f8c6a0_d59083b02ef74bdf8122e0c51454aa71 + 2*fdefb3e070394d5cbf22820ac0ab7b00_c06e7e291fdb4cb6b5ec0b9abbabf1bf